<a href="https://colab.research.google.com/github/franklinwillemen/Sherlock-Hemlock-Graph-Based-NER/blob/main/data_build_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import transformers as t
import datasets
from datasets import load_dataset

import torch
import torch_geometric
from torch_geometric.data import Data

import numpy as np
import pandas as pd

import spacy
from spacy.util import minibatch

import flair
from flair.data import Sentence
from flair.models import SequenceTagger

from sklearn.preprocessing import LabelEncoder

In [2]:
""" articles = load_dataset("maastrichtlawtech/bsard", data_files="articles_fr.csv", split="train")
df_art = pd.DataFrame(articles.to_pandas())
df_art.head() """

articles = pd.read_csv("bsard_articles_extra.csv",index_col=0)

In [3]:
articles.columns

Index(['id', 'reference', 'article', 'law_type', 'code', 'book', 'part', 'act',
       'chapter', 'section', 'subsection', 'description',
       'preprocessed_article', 'word_count', 'sentence_count'],
      dtype='object')

In [4]:
snlp = spacy.load("fr_dep_news_trf")
fnlp = SequenceTagger.load("flair/ner-french")

2023-04-29 00:08:07,770 SequenceTagger predicts: Dictionary with 19 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, S-PER, B-PER, E-PER, I-PER, S-MISC, B-MISC, E-MISC, I-MISC, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


In [13]:
def extract_pos_dep_head(sentence, nlp):
    try:
        doc = nlp(sentence)
        pos = [token.pos_ for token in doc]
        dep = [token.dep_ for token in doc]
        heads = [token.head.i for token in doc]
    except:
        pos, dep, heads = [], [], []
    return pos, dep, heads

def extract_ner(sentence, tagger):
    try:
        sent = Sentence(sentence)
        tagger.predict(sent)
        ner = [str(ent) for ent in sent.get_spans('ner')]
    except:
        ner = []
    return ner

def process_article(article, nlp, tagger):
    # Split article into sentences
    doc = nlp(article)
    sentences = [sent.text for sent in doc.sents]
    
    # Extract features for each sentence
    pos, dep, heads, ner = [], [], [], []
    for sentence in sentences:
        pos_s, dep_s, heads_s = extract_pos_dep_head(sentence, nlp)
        ner_s = extract_ner(sentence, tagger)
        
        pos.append(pos_s)
        dep.append(dep_s)
        heads.append(heads_s)
        ner.append(ner_s)
    
    return pos, dep, heads, ner

def process_articles(df, nlp, tagger):
    df['pos'], df['dep'], df['heads'], df['ner'] = zip(*df['article'].apply(process_article, args=(nlp, tagger)))
    return df

In [14]:
df_subset = articles[:64]

In [15]:
df = process_articles(df_subset, snlp, fnlp)

KeyboardInterrupt: 

In [ ]:
df.to_csv("bsard_articles_extra.csv")

In [ ]:
df_train.to_csv("../local_datasets/mapa_plus/mapa_plus_train.csv", index = False)
df_test.to_csv("../local_datasets/mapa_plus/mapa_plus_test.csv", index = False)
df_val.to_csv("../local_datasets/mapa_plus/mapa_plus_val.csv", index = False)